In [1]:
import json
import pandas as pd
import nbimporter

In [2]:
import encryption

Importing Jupyter notebook from encryption.ipynb


### IMPORT DATA

In [3]:
dataframe = pd.read_csv('data/all-files-ignore-code-kernel-fix.csv', index_col='Unnamed: 0')

In [4]:
data = dataframe.to_dict()

### CREATE BATCH OF LOG EVENTS

In [5]:
def create_batch(hashes, data):
    
    # DATA CONTAINER
    container = {}
    
    # DATAFRAME COLUMNS
    cols = data.keys()
    
    # LOOP THROUGH
    for target_hash in hashes:
        
        # CREATE ENTRY
        container[target_hash] = {}
        
        # LOOP THROUGH COLUMNS
        for col in cols:
            
            # ADD PROPERTY
            container[target_hash][col] = data[col][target_hash]

    return container

In [6]:
batch = create_batch([
    '53eac99acba6c33cfabdf631e4c7a0196f2e1647a0303190375ea680190510ed',
    '81759ec7de5efa9657b37800e1ec317692fe41c67ca7de759b23fed45015a931',
    '30129d78176e0d7a3e9fe5948b2df6bd3b4afd183f6b47f4c67a585239bf8fd7',
    'c4ec89aae91fea6f4c675192f0d774c5796e101500d50d7db77bcac276ee0a16'
], data)

### AES ENCRYPT BATCH EVENT

In [7]:
sym_data, sym_params = encryption.aes_encrypt(batch)

In [8]:
sym_params

b'{"password": "wX7PBtTSpWiiOm9e", "init_vector": "LinC71sLI_fgLyU5"}'

### FETCH RSA PUBLIC KEY -- TEMPORARY

In [9]:
raw_rsa_pubkey, raw_rsa_privkey = encryption.generate_rsa_keys()

In [10]:
# SANITIZE TO SIMULATE BLOCKCHAIN FORMAT
sanitized = encryption.sanitize_key(raw_rsa_pubkey)

In [11]:
# CONVERT TO RSA FORMAT
rsa_pubkey = encryption.serialize_key(sanitized)

### RSA ENCRYPT AES PARAMS

In [12]:
asym_params = encryption.rsa_encrypt(sym_params, rsa_pubkey)

In [13]:
asym_params

b'-\xf8C\xc8_\x05\xb9\xcf\xde\xd3\xdf\xff\xfa\xd99\xca\x81\xab\xd7\x82\xa0[\x01u5l\xd13M3\xe1|\x83"\xf5\x1c\xdd\xa4G\x07\xd00}\xc9\xa5\xbdC\xeb\xa4\x82\xc1\x07AV\xbd\xa0\xd8o\xdb#\xc7e\xf4AH@\x07\xe8n\xed\xfa\xdd\xda\x1b\xc6\xc9\xd6~\x17$\xe8\xd9\x883\xfd\x04\x16\x9c\xa2.\xb1!\xcc\xefu\x15\xc39H\xe9\xd7\xb3\x0fD\xe8\xb9e\x99\xfdZ\xc2\xd6\xa0\xd8\xf1|\xe1g\x83Ag\x1d\xd0\xe7:\x0c\xf3]H\x93\xb9\xe6zq~\xd7\xca\xca@H\x8aO\xc9\xa4<\\\x02MDv\x95\xcc\xf2\x16\xb7i\x1d -2\xc7!\xab5ma\xd5\x9d\xda\x7fs\xe1\xdfs\xf1\xf3\xa49\x91\x95\xe2Gx\xber\xd0X\x96Or\xe36`<\xca7v\xa2\x1d\xfc\x9eW\xcaE\x94~\x9cuM\xa4ha\x1c\xb2\x0bd\x11q8q\x03\xd1\xcds\x01\xfe\xaeT7\x92\xf7\'z4L\x19\xbd\x86\x85\x8a\xdbH\xda\xff\xba\xbd\xc9m\xe2A\xb8\xc7\x88`\xca%'

#### ----------------- IOT PROCESS STOPS -----------------

### RSA DECRYPT THE AES SYMKEY PARAMS

In [14]:
decrypt_params = encryption.rsa_decrypt(asym_params, raw_rsa_privkey)

In [15]:
decrypt_params

{'password': 'wX7PBtTSpWiiOm9e', 'init_vector': 'LinC71sLI_fgLyU5'}

### AES DECRYPT THE DATA

In [16]:
decrypt_data = encryption.aes_decrypt(sym_data, decrypt_params)

In [17]:
decrypt_data

{'53eac99acba6c33cfabdf631e4c7a0196f2e1647a0303190375ea680190510ed': {'occurrence': 4,
  'module': 'cron',
  'code': '325',
  'message': '(CRON) INFO (pidfile fd = 3)',
  'timestamp': 1613389600},
 '81759ec7de5efa9657b37800e1ec317692fe41c67ca7de759b23fed45015a931': {'occurrence': 6,
  'module': 'CRON',
  'code': '15203',
  'message': '(CRON) info (No MTA installed, discarding output)',
  'timestamp': 1613646001},
 '30129d78176e0d7a3e9fe5948b2df6bd3b4afd183f6b47f4c67a585239bf8fd7': {'occurrence': 144,
  'module': 'CRON',
  'code': '6126',
  'message': '(root) CMD ( cd / && run-parts --report /etc/cron.hourly)',
  'timestamp': 1613625421},
 'c4ec89aae91fea6f4c675192f0d774c5796e101500d50d7db77bcac276ee0a16': {'occurrence': 6,
  'module': 'CRON',
  'code': '15207',
  'message': '(root) CMD (./home/wickstjo/scripts/ropsten.sh)',
  'timestamp': 1613646001}}